<a href="https://colab.research.google.com/github/rsin46/pegasus-demo/blob/master/Pegasus_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MODEL: gigaword for gigaword
#        cnn_dailymail for CNN Dailymail
MODELNAME = 'aeslc'

In [2]:
!git clone http://github.com/rsin46/pegasus-demo.git
!git clone http://github.com/google-research/pegasus.git

Cloning into 'pegasus-demo'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 17 (delta 0), reused 17 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.
Cloning into 'pegasus'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 353.09 KiB | 1.02 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir /content/pegasus-demo/model
!pip install sentencepiece
!pip install tensorflow_transform

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
     |████████████████████████████████| 1.1MB 5.8MB/s 
     |████████████████████████████████| 327kB 8.6MB/s 
     |████████████████████████████████| 516.2MB 15kB/s 
     |████████████████████████████████| 2.0MB 45.8MB/s 
     |████████████████████████████████| 112kB 59.6MB/s 
     |████████████████████████████████| 8.3MB 49.0MB/s 
     |████████████████████████████████| 460kB 46.6MB/s 
     |████████████████████████████████| 3.0MB 51.2MB/s 
     |███████████

In [ ]:
if MODELNAME == 'gigaword':
  !cp '/content/gdrive/My Drive/Models/gigaword.tar.xz' '/content/pegasus-demo/model/gigaword.tar.xz'
  !tar -xvf '/content/pegasus-demo/model/gigaword.tar.xz' -C '/content/pegasus-demo/model/' 
elif MODELNAME == 'cnn_dailymail':
  !cp '/content/gdrive/My Drive/Models/cnn_dailymail.xz' '/content/pegasus-demo/model/cnn_dailymail.xz'
  !tar -xvf '/content/pegasus-demo/model/cnn_dailymail.xz' -C '/content/pegasus-demo/model/'
elif MODELNAME == 'aeslc':
  !cp -r '/content/gdrive/My Drive/Models/aeslc/' '/content/pegasus-demo/model/'


In [ ]:
%cd /content/pegasus/
if MODELNAME == 'aeslc':
  import itertools
  import os
  import time

  from absl import logging
  from pegasus.data import infeed
  from pegasus.params import all_params  # pylint: disable=unused-import
  from pegasus.params import estimator_utils
  from pegasus.params import registry
  import tensorflow as tf
  from pegasus.eval import text_eval
  from pegasus.ops import public_parsing_ops
  import pandas as pd
  from random import choice
  from tensorflow.python.estimator.export import export
  tf.enable_eager_execution()
  # import tensorflow_transform as tft
  data_name = MODELNAME
  import tensorflow_transform as tft

  master = ""
  model_dir = "./ckpt/pegasus_ckpt/%s"%data_name
  use_tpu = False
  iterations_per_loop = 1000
  num_shards = 1
  param_overrides = "vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6"

  eval_dir = os.path.dirname(model_dir)
  checkpoint_path = model_dir
  checkpoint_path = tf.train.latest_checkpoint(checkpoint_path )
  params = registry.get_params('%s_transformer'%data_name)(param_overrides)
  pattern = params.dev_pattern
  input_fn = infeed.get_input_fn(params.parser, pattern,
                                      tf.estimator.ModeKeys.PREDICT)
  parser, shapes = params.parser(mode=tf.estimator.ModeKeys.PREDICT)
  RAW_DATA_FEATURE_SPEC = dict([("inputs", tf.io.FixedLenFeature(shapes['inputs'], tf.int64)), 
                                ('targets', tf.io.FixedLenFeature(shapes['targets'], tf.string))])

  raw_feature_spec = RAW_DATA_FEATURE_SPEC.copy()
  raw_feature_spec.pop('targets')
  # raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  #         raw_feature_spec, default_batch_size=0)
  def serving_input_fn():
      raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
          raw_feature_spec, default_batch_size=1)
      serving_input_receiver = raw_input_fn()

      raw_features = serving_input_receiver.features
      return tf.estimator.export.ServingInputReceiver(
          raw_features, serving_input_receiver.receiver_tensors)

  print(tf.executing_eagerly())
  estimator = estimator_utils.create_estimator(master, 
                                              model_dir,
                                              use_tpu,
                                              iterations_per_loop,
                                              num_shards, params, include_features_in_predictions=False)

  output = estimator.export_saved_model(
      "model/", serving_input_fn
  )

In [ ]:
%cd /content/pegasus-demo
# Initialising variables

TEXT = 'thermo.txt'
OUT_TEXT = '/content/gdrive/My Drive/Models/' + TEXT

def append(line):
    with open(OUT_TEXT, 'a', buffering=1) as redf:
        redf.write(line)

/content/pegasus-demo


In [ ]:
import text_eval
import public_parsing_ops
import tensorflow as tf
import numpy as np
import datetime

_SPM_VOCAB = 'ckpt/c4.unigram.newline.10pct.96000.model'
encoder = public_parsing_ops.create_text_encoder("sentencepiece",
                                                 _SPM_VOCAB)
shapes = {
    'cnn_dailymail': (1024, 128),
    'gigaword':(128, 32),
    'aeslc':(512, 32)
}

model_dir = {
    'cnn_dailymail': 'model/1595037231/',
    'gigaword': 'model/1595095633/',
    'aeslc': 'model/aeslc'
}

imported = tf.saved_model.load(model_dir[MODELNAME], tags='serve')


text = open(TEXT, "r", encoding="utf-8").read()


shape,_ = shapes[MODELNAME]

input_ids_raw = encoder.encode(text)
input_ids_split = list(zip(*([iter(input_ids_raw)]*shape)))

out_file = open(OUT_TEXT,"a")
current_time = datetime.datetime.now()
append(current_time.strftime("%d-%b-%Y (%H:%M:%S.%f)") + '\n')

for input_ids in input_ids_split:
  inputs = np.zeros(shape)
  idx = len(input_ids)
  if idx>shape: idx =shape

  inputs[:idx] = input_ids[:idx]

  example = tf.train.Example()

  example.features.feature["inputs"].int64_list.value.extend(inputs.astype(int))
  output = imported.signatures['serving_default'](examples=tf.constant([example.SerializeToString()]))
  prediction = text_eval.ids2str(encoder, output['outputs'].numpy(), None)
  print("\nPREDICTION >> ", prediction)
  append(prediction + '\n')


OSError: ignored

In [ ]:
import text_eval
import public_parsing_ops
import tensorflow as tf
import numpy as np

_SPM_VOCAB = 'ckpt/c4.unigram.newline.10pct.96000.model'
encoder = public_parsing_ops.create_text_encoder("sentencepiece",
                                                 _SPM_VOCAB)
shapes = {
    'cnn_dailymail': (1024, 128),
    'gigaword':(128, 32)
}

model_dir = {
    'cnn_dailymail': 'model/1595037231/',
    'gigaword': 'model/1595095633/'
}

imported = tf.saved_model.load(model_dir[MODELNAME], tags='serve')
example = tf.train.Example()

text = open(TEXT, "r", encoding="utf-8").read()


shape,_ = shapes[MODELNAME]

input_ids = encoder.encode(text)


inputs = np.zeros(shape)
idx = len(input_ids)
if idx>shape: idx =shape

inputs[:idx] = input_ids[:idx]

example.features.feature["inputs"].int64_list.value.extend(inputs.astype(int))
output = imported.signatures['serving_default'](examples=tf.constant([example.SerializeToString()]))

print("\nPREDICTION >> ", text_eval.ids2str(encoder, output['outputs'].numpy(), None))

In [ ]:
a = tf.constant([example.SerializeToString()])
b = tf.constant([example.SerializeToString()])
tf.tile(a, b)

In [ ]:
tf.repeat(a,2)